This notebook contains the pipeline for the LoF variants extraction from UKBB 450k data. 
It is an approximation of the method explained in the [paper](https://www.nature.com/articles/s41586-022-04549-9). 

This notebook should be placed in UKBB Research Analysis Platform. 

The cell that contains `chromosome` variable should be tagged  as explained [here](https://papermill.readthedocs.io/en/latest/usage-parameterize.html#designate-parameters-for-a-cell). 

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Import-&amp;-preparation" data-toc-modified-id="Import-&amp;-preparation-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Import &amp; preparation</a></span></li><li><span><a href="#Load-and-filter-variants-by-bed" data-toc-modified-id="Load-and-filter-variants-by-bed-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Load and filter variants by bed</a></span></li><li><span><a href="#VEP" data-toc-modified-id="VEP-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>VEP</a></span></li><li><span><a href="#Filter-missing-genotypes" data-toc-modified-id="Filter-missing-genotypes-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Filter missing genotypes</a></span></li><li><span><a href="#How-to-run-it" data-toc-modified-id="How-to-run-it-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>How to run it</a></span></li></ul></div>

# Import & preparation

Prior to running the notebook, the following data should be uploaded to the RAP:
1. The GRCh38 coordinates of the targeted regions `xgen_plus_spikein.GRCh38.bed` from https://biobank.ndph.ox.ac.uk/ukb/refer.cgi?id=3803 .

2. The GRCh38 coordinates of the genes with s-het>=0.15 `high_s_het_gene_list_10bp.bed` from the previous step.

3. List of unrelated samples `unrelated_samples.txt`. 

In this section we also download `pvcf_blocks.txt` as the data for each chromosome is splitted across several files, marked by `block_id`. 

In [ ]:
import pyspark
import hail as hl
import pandas as pd

# initialize Spark and Hail contexts
sc = pyspark.SparkContext()
spark = pyspark.sql.SparkSession(sc)

hl.init(sc=sc)

The following cell should be tagged as explained [here](https://papermill.readthedocs.io/en/latest/usage-parameterize.html#designate-parameters-for-a-cell).

It will be used as a command line argument for a papermill tool.

In [ ]:
# this cell is tagged parameters (will act as a command line argument)
chromosome = 21

In [ ]:
# load bed for target sequencing region
target_bed_path = 'file:///.../xgen_plus_spikein.GRCh38.bed' # should be specified

bed_target = hl.import_bed(target_bed_path, reference_genome='GRCh38')

We load file, that contains bed regions of the genes with `s_het > 0.15`. We will use it for the filtration. This file was generated on the previous step `1_create_gene_list_turbo.ipynb` and copied to `Uploaded data` folder.

In [ ]:
# load bed for genes with s_het>0.15
contig_recoding = {str(i): 'chr' + str(i) for i in range(1, 23)}
contig_recoding.update({'X': 'chrX', 'Y': 'chrY', 'MT': 'chrM'})

genes_bed_path = 'file:///.../high_s_het_gene_list_10bp.bed'  # should be specified

bed_genes = hl.import_bed(genes_bed_path, reference_genome='GRCh38', contig_recoding=contig_recoding)

We load the list of samples, created by the main pipeline, that contains **unrelated** participants. 

In [ ]:
# load samples that should be used in analysis and doesnt contain related samples

unrelated_samples_path = ... # should be specified

with open(unrelated_samples_path, 'r') as f:
    samples_list = f.readlines()
    
samples_list = [sample_id.strip() for sample_id in samples_list]
n_samples = len(samples_list)

print (n_samples)

samples_list[:3]

pVCF files are stored in blocks, so that every chromosome is divided into several files. Here we load information about the blocks created for every chromosome. 

In [ ]:
# read pvcf blocks file
pvcf_blocks = pd.read_csv("https://biobank.ctsu.ox.ac.uk/crystal/ukb/auxdata/pvcf_blocks.txt", 
                          sep='\t', header=None)
pvcf_blocks.columns = ['row_id', 'chrom', 'block_id', 'start', 'end']

chromosome_pvcf_blocks = pvcf_blocks[pvcf_blocks['chrom'].astype(int) == int(chromosome)]['block_id'].tolist()

print (len(chromosome_pvcf_blocks))

# Load and filter variants by bed

In this section we:  

1. Load all gvcfs associated with a `chromosome`.

2. Leave only target regions and genes of interest (`s_het >= 0.15`) as described [here](https://hail.is/docs/0.2/guides/genetics.html#from-a-ucsc-bed-file).

3. Concatenate all gvcfs into one. 

In [ ]:
import sys

# we define a loading function
def load_filter_gvcf(gvcf_path):
    """
        Reads gvcf stored in `gvcf_path` and 
        filters target regions and genes of interest
        defined in `bed_genes` and `bed_target`.
    """
    print (f"load {gvcf_path}")
    sys.stdout.flush()
    
    gvcf = hl.methods.import_vcf(gvcf_path, 
                                 force_bgz=True, 
                                 reference_genome='GRCh38', 
                                 array_elements_required=False,
                                 block_size=32)
    
    # this is described here https://hail.is/docs/0.2/guides/genetics.html#from-a-ucsc-bed-file 
    gvcf = gvcf.filter_rows(hl.is_defined(bed_genes[gvcf.locus]))
    gvcf = gvcf.filter_rows(hl.is_defined(bed_target[gvcf.locus]))
    
    return gvcf

# function, that loads several gVCFs
def load_concatenate_gvcfs(gvcf_paths):
    """
        Reads all gvcfs defined in `gvcf_paths` list
    """
    gvcfs = [load_filter_gvcf(gvcf_path) for gvcf_path in gvcf_paths]
    
    return gvcfs

In [ ]:
# generate all gvcf paths for a given chromosome
gvcf_paths = [
    f'file:///.../'
    f'Population level exome OQFE variants, pVCF format - final release/ukbXXXXX_c{chromosome}_b{idx}_v1.vcf.gz' for idx in chromosome_pvcf_blocks
] # should be specified

# load all gvcfs for a given chromosome
vcf = load_concatenate_gvcfs(gvcf_paths)

# concatenate all vcfs together, may trigger a shuffle if partitions overlap.
full_vcf = hl.MatrixTable.union_rows(*vcf, _check_cols=False)

# VEP

In this section we : 
1. Annotate variants with VEP
    a. Here we use `json.vep` configuration file, the same file can be found next to the script file.

    b. `transcript_consequences` field will contain all consequences mapped to the variants in the location. 

2. Leave only variants, that are high quality LoF (based on lottee) in any canonical transcript: 

    a. Iterate over all consequences in `transcript_consequences`, select canonical transcripts and parse consequence term. 

    b. For LoF consequence check if it is high quality or low quality as annotated by Loftee. 

    c. Leave only locuses with LoF high-quality variants as annotated in canonical transcript. 

In [ ]:
# function, that defines LoF variant
def is_lof(consequence_terms):
    is_lof = (
        consequence_terms.contains("splice_acceptor_variant") | 
        consequence_terms.contains("splice_donor_variant") | 
        consequence_terms.contains("stop_gained") | 
        consequence_terms.contains("frameshift_variant")
    )
    
    return is_lof


# function, that parses synonimous, LoF, missense protein consequences in canonical transcript
# and otherwise returns missing value
def process_transcript_consequences(transcript_consequence):    
    return hl.if_else(transcript_consequence.canonical == 1, 
                      hl.case()
                        .when(transcript_consequence.consequence_terms.contains("synonymous_variant"),
                              hl.struct(allele_num=transcript_consequence.allele_num, consq='synonymous'))
                        .when(is_lof(transcript_consequence.consequence_terms) & (transcript_consequence.lof == 'HC'),
                              hl.struct(allele_num=transcript_consequence.allele_num, consq='lof_hc'))
                        .when(is_lof(transcript_consequence.consequence_terms) & (transcript_consequence.lof == 'LC'),
                              hl.struct(allele_num=transcript_consequence.allele_num, consq='lof_lc'))
                        .when(transcript_consequence.consequence_terms.contains("missense_variant"),
                              hl.struct(allele_num=transcript_consequence.allele_num, consq='missense'))
                        .or_missing(), 
                      hl.missing('struct{allele_num: int32, consq: str}'))


In [ ]:
# annotate gVCF with vep
annotated_vcf = hl.vep(full_vcf, "file:///.../json.vep")

# parse consequences 
annotated_vcf = annotated_vcf.annotate_rows(vep_canonical_consequences=hl.map(process_transcript_consequences, 
                                                                              annotated_vcf.vep.transcript_consequences))

# leave only variants that have high quality LoF in at least one canonical transcript
lof_vcf = annotated_vcf.filter_rows(hl.any(hl.map(lambda x: x.consq == 'lof_hc', 
                                                  annotated_vcf.vep_canonical_consequences)))

# Filter missing genotypes

In this section we filter out variants based on the quality as it wass described in the [paper](https://www.nature.com/articles/s41586-022-04549-9):

1. Leave only unrelated samples

2. Calculate p-value for twwi-sided binomial test (number of supporting alleles, total coverage, p=0.5) for only heterozygous variants 

3. Leave only samples with `DP >=7`, `GQ >=20` and `p-value > 0.001`(if applicable)

4. Add information about number of non-empty genotypes per row, and filter locuses with more that 50& missing genotypes. 

5. Save resulting vcf file

In [ ]:
# leave only unrelated samples
lof_vcf = lof_vcf.filter_cols(hl.array(samples_list).contains(lof_vcf.s))

In [ ]:
# calculate binomial test p-value for heterozygous variant
lof_vcf = lof_vcf.annotate_entries(pval=hl.if_else(lof_vcf.GT.is_het(),
                                                   hl.binom_test(lof_vcf.AD[lof_vcf.GT[1]], lof_vcf.DP, 0.5, 'two-sided'), 
                                                   1.))

In [ ]:
# replace genotypes with low reliability to missing

lof_vcf = lof_vcf.filter_entries((lof_vcf.DP >= 7) & (lof_vcf.GQ >= 20) & (lof_vcf.pval > 0.001))

In [ ]:
# filter variants with more than 50% missing genotypes

lof_vcf = lof_vcf.annotate_rows(non_empty=hl.agg.count())
lof_vcf = lof_vcf.filter_rows(lof_vcf.non_empty >= n_samples/2.)

In [ ]:
# save results as vcf
hl.export_vcf(lof_vcf, f'file:///.../chr{chromosome}_lof.vcf.bgz')

cleanup log files:

In [ ]:
!rm -rf /.../hail*

# How to run it

In order to run it for a specific chromosome the following things should be done:

1. Copy `2_data_collection_paper_rap` to UKBB RAP

2. For every chromosome submit the following command:


`my_cmd="papermill 2_data_collection_paper_rap.ipynb Data_collection_paper_chrNN_output.ipynb -p chromosome NN"`

`dx run dxjupyterlab_spark_cluster --instance-type=mem2_ssd1_v2_x16 --instance-count=5 --priority=low --name="Run analysis chrNN" -icmd="$my_cmd" -iduration=480 -iin="../2_data_collection_paper_rap.ipynb" -ifeature="HAIL-0.2.78-VEP-1.0.3"`



Commands should be submitted from a local machine with installed CLI for DNAnexus.